## push test

In [ ]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [ ]:
C = ry.Config()
D = C.view()
C.addFile("model.g")
Xstart = C.getFrameState()
S = C.simulation(ry.SimulatorEngine.bullet, True)

In [ ]:
for k in range (1):
    #restart from same state multiple times
    S.setState(Xstart)

    tau = .01
    box = C.frame("box")

    for t in range(1000):
        

        if t%10 == 0:
            S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

        time.sleep(0.01)

        #some good old fashioned IK
        q = C.getJointState();
        [y,J] = C.evalFeature(ry.FS.positionDiff, ["gripper", "ring"])
        y = y * .003 / np.linalg.norm(y)
        q = q - J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ y
        C.setJointState(q)
        S.step(q, tau,  ry.ControlMode.position)
        

#         if t%100 == 0:
#             p = box.getPosition()
#             p[0] += .05
#             p[2] += .2
#             box.setPosition(p)
#             S.setState(C.getFrameState())

## grasp test

In [ ]:
import os
os._exit(0)

In [ ]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [ ]:
C = ry.Config()
D = C.view()
C.addFile("model.g")
Xstart = C.getFrameState()
C.selectJoints(["finger1", "finger2"], True)

In [ ]:
#S = C.simulation(ry.SimulatorEngine.physx, True)
S = ry.Simulation(C, ry.SimulatorEngine.bullet, True)

In [ ]:
tau = .01
t=0
#S.setState(Xstart, [])

while True:
    t = t+1
    
    if t%10 == 0:
        S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    time.sleep(0.01)

    q = S.get_q()

    #some good old fashioned IK
    if t > 40 and t<=300:
        #q = C.getJointState()
        [y,J] = C.evalFeature(ry.FS.oppose, ["finger1", "finger2", "ring4"])
        y = y * min(.008/np.linalg.norm(y), 1.)
        q = q - J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ y
        
    if t==300:
        S.closeGripper("gripper")
        
    if S.getGripperIsGrasping("gripper"):
        [y,J] = C.evalFeature(ry.FS.position, ["gripper"]);
        q = q - J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ [0.,0.,-2e-4]

    if t==900:
        S.openGripper("gripper")
    
    if t>1000 and S.getGripperWidth("gripper")>=.02:
        break

    S.step(q, tau, ry.ControlMode.position)


In [ ]:
S=0
D=0
C=0

## Stack of Blocks Test

In [ ]:
import os
os._exit(0)

In [ ]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [ ]:
C = ry.Config()
D = C.view()
for i in range(7):
    obj = C.addFrame("obj"+str(i))
    obj.setShape(ry.ST.ssBox, [.2,.2,.2,.02])
    obj.setPosition([0,0,1+0.25*i])
    obj.setMass(1.)

C.addFile("../../scenarios/pandasTable.g")

In [ ]:
S = ry.Simulation(C, ry.SimulatorEngine.bullet, True)
tau = .01
box = C.frame("box")

for t in range(400):
    S.step([], tau, ry.ControlMode.none)
    time.sleep(0.01)

## Friction Test

In [ ]:
import os
os._exit(0)

In [ ]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [ ]:
C = ry.Config()
D = C.view()
for i in range(10):
    obj = C.addFrame("obj"+str(i))
    obj.setShape(ry.ST.ssBox, [.1,.1,.1,.01])
    obj.setPosition([.15*(i-5),0,1])
    obj.setMass(.2)
    obj.addAttribute("friction", .02*i)

C.addFile("../../scenarios/pandasTable.g")
C.frame("table").setQuaternion([1., -.1, 0., 0.])

In [ ]:
S = ry.Simulation(C, ry.SimulatorEngine.bullet, True)
tau = .01
box = C.frame("box")

for t in range(400):
    S.step([], tau, ry.ControlMode.none)
    time.sleep(0.01)

## Spline Execution Test

In [ ]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time

C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
komo = C.komo_path(1., 20, 5., True)
komo.addObjective([1.], ry.FS.distance, ["R_finger2_coll", "table"], ry.OT.eq, [1e2])

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e1], order=1)
komo.optimize()
komo.view_play(False, 0.1)

path_q = komo.getPath_qOrg()
path_q.shape

In [ ]:
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/pandasTable.g")

S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.setMoveto(path_q, 5.)

tau = 0.01
for t in range(1000):
    S.step([], tau, ry.ControlMode.spline)
    if t%100 == 0:
        print(t*tau, "timeToGo: ", S.getTimeToMove())
        
    time.sleep(0.01)
    if S.getTimeToMove() < 0.:
        break

# noPenetration Imp Test

In [ ]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time

C0 = ry.Config()
C0.addFile("../../scenarios/pandasTable.g")

stick = C0.addFrame("stick")
stick.setShape(ry.ST.capsule, [0.5, 0.025])
stick.setPosition([0,0,.68])
stick.setQuaternion([1,0,1,0])
stick.setContact(1)
stick.setMass(0.1)

tip = C0.addFrame("stickTip", "stick")
tip.setShape(ry.ST.marker, [0.1])
tip.setRelativePosition([0,0,-0.25])

IK = C0.komo_IK(False);
IK.addObjective([], ry.FS.positionRel, ["R_gripperCenter", "stick"], ry.OT.eq, [1e2], [0, 0, 0.2]);
IK.addObjective([], ry.FS.scalarProductXZ, ["R_gripperCenter", "stick"], ry.OT.eq, [1e2]);
IK.addObjective([], ry.FS.vectorZ, ["R_gripperCenter"], ry.OT.eq, [1e2], [0,0,1]);
IK.optimize()
IK.view(False)

In [ ]:
C0.setFrameState(IK.getFrameState(0))
tau, qdim = 0.01, C0.getJointDimension()
for activate in [False, True]:
    C = ry.Config()
    C.copy(C0)
    S = C.simulation(ry.SimulatorEngine.bullet, True)
    
    if activate:
        S.addImp(ry.ImpType.noPenetrations)
        
    S.closeGripper("R_gripper");
    while not S.getGripperIsGrasping("R_gripper"):
        time.sleep(tau)
        S.step([], tau,  ry.ControlMode.none);
        
    for _ in range(100):
        time.sleep(tau)
        [y, J] = C.evalFeature(ry.FS.position, ["R_gripper"])
        vel = np.linalg.inv(J.T@J + 1e-2*np.eye(qdim)) @ J.T @ np.array([0,0,.3])
        S.step(vel, tau, ry.ControlMode.velocity)

    for _ in range(200):
        time.sleep(tau)
        [y, J] = C.evalFeature(ry.FS.position, ["stickTip"])
        vel = np.linalg.inv(J.T@J + 1e-2*np.eye(qdim)) @ J.T @ np.array([0,0,-.5])
        S.step(vel, 0.01, ry.ControlMode.velocity)
